In [ ]:
import numpy as np
import time

import matplotlib.pyplot as plt

from astropy.table import Table, vstack, hstack
from astropy.coordinates import SkyCoord
import astropy.units as u

from hetdex_api.shot import get_fibers_table
from hetdex_api.survey import FiberIndex

# Query a single coordinate:

In [ ]:
coord = SkyCoord(ra=189.29668*u.deg, dec=62.241787*u.deg)

In [ ]:
# Intiate the FiberIndex class from hetdex_api.survey:
F = FiberIndex()

#help(F.query_region)

FiberIndex.Query_region() returns an astropy table of all fibers within the the aperture defined. Default is 3.5 arcsec radius. amp_flag, gal_flag, meteor flag populate whether the fiber would make it into the current catalog. 1 is good, 0 is removed. 'flag' combines the three flags

In [ ]:
# This example was observed in multiple observations so there are many associated fibers
fibtab = F.query_region(coord, radius=3.5*u.arcsec)
fibtab.show_in_notebook()

In [ ]:
# close the FiberIndex class (and associated open h5 files) when done
F.close()

# To grab the fibers use Extract

In [ ]:
from hetdex_api.extract import Extract

In [ ]:
obslist = np.unique(fibtab['shotid'])

In [ ]:
E = Extract()
E.load_shot(obslist[0])

In [ ]:
ifux, ifuy, xc, yc, ra, dec, spec, spece, mask, fiberid_array, mf_array = E.get_fiberinfo_for_coord(coord, return_fiber_info=True)

In [ ]:
np.shape(spec) # 1st dim is fiber dimension, 2nd is wavelength

In [ ]:
%matplotlib inline

In [ ]:
# Here is every fiber in the 3.5 arcsec aperture
plt.figure(figsize=(10,5))
for i in np.arange(np.shape(spec)[0]):
    plt.plot(E.wave, spec[i], label='FiberID is {}'.format(fiberid_array[i]))
    
#plt.legend()

# Query a table with < 10 K Sources

In [ ]:
# Open the pared down SDSS AGN catalog. See below for how this was done.

In [ ]:
sdss_agn = Table.read('sdss-dex-agn.fits')
agn_coords = SkyCoord(ra = sdss_agn['RA'], dec= sdss_agn['DEC'])

In [ ]:
from multiprocessing import Pool

In [ ]:
def get_fiber_table(coord):
    F = FiberIndex()
    tab = F.query_region(coord, radius=3.5*u.arcsec)
    F.close()
    return tab

In [ ]:
t0 = time.time()
p = Pool()
res = p.map(F.query_region, agn_coords)
p.close()
t1 = time.time()

print((t1-t0)/60)

In [ ]:
t0 = time.time()
p = Pool()
res = p.map(get_fiber_table, agn_coords)
p.close()
t1 = time.time()

print((t1-t0)/60)

In [ ]:
res[0]

The output from multiprocessing will be a list of the astropy table object printed above for each coordinate. Loop through the results and do what you like with the data or change the function to do it faster with multiprocessing. For example this will go through the AGN catalog and return a list of shotid's for a given coordinate and will provide the net 'flag' value for that aperture

In [ ]:
def get_nobs(coord):
    
    F = FiberIndex()
    fib_table = F.query_region(coord, radius=3.5*u.arcsec)
    F.close()
    
    # check how many fibers in aperture for each shotid
    shotlist, nfib_list = np.unique(fib_table['shotid'], return_counts=True)
    n_obs = np.size(shotlist)
    
    flag_list = []
    for shot in shotlist:
        flag_list.append( int(np.all(fib_table['flag'][fib_table['shotid']==shot])) )
    n_obs_good = np.sum(flag_list)
    
    return n_obs, n_obs_good, list(shotlist), list(nfib_list), list(flag_list)

In [ ]:
get_nobs(agn_coords[10])

In [ ]:
t0 = time.time()
p = Pool()
res = p.map(get_nobs, agn_coords)
p.close()
t1 = time.time()

print((t1-t0)/60)

In [ ]:
n_obs = []
n_obs_good = []
shots_for_src = []
nfib_for_src = []
shotflags = []

for r in res:
    n_obs.append(r[0])
    n_obs_good.append(r[1])
    shots_for_src.append(r[2]) 
    nfib_for_src.append(r[3]) 
    shotflags.append(r[4])

In [ ]:
print('{} sources have at least one good observation'.format(np.sum( np.array(n_obs_good) > 0)))

# Query a very large catalog by loading the full fibers table. Use astropy search_around_sky for faster querying through kdtree. This is best done on stampede2 


In [ ]:
#https://docs.astropy.org/en/stable/coordinates/matchsep.html#searching-around-coordinates

In [ ]:
sdss_agn = Table.read('DR14Q_v4_4.fits')
agn_coords = SkyCoord(ra = sdss_agn['RA'], dec= sdss_agn['DEC'])

In [ ]:
# If you set load_fiber_table=True, the full fiber table will load. This takes some time,
# takes loads of memory. Best done on stampede2 and takes 2 minutes
F = FiberIndex(load_fiber_table=True)

In [ ]:
# The full array of fiber coordinates are here:
F.coords

In [ ]:
sel_good = np.isfinite(F.coords.ra.value) # some values are NaN so this gets rid of them

In [ ]:
t0 = time.time()
idxagn, idxF, sep2d, dist3d = F.coords[sel_good].search_around_sky(agn_coords, seplimit=3.5*u.arcsec)
t1 = time.time()
print(t1-t0)

In [ ]:
# save table for AGN that have at least one fiber coverage
sdss_agn[np.unique(idxagn)].write('sdss-dex-agn.fits', overwrite=True)

In [ ]:
sdss_agn[np.unique(idxagn)]

In [ ]:
# Combined the tables with hstack

In [ ]:
matched_fiber_table = hstack([sdss_agn[idxagn], F.fiber_table[sel_good][idxF]])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#quick check to make sure coordinates matched
plt.scatter( matched_fiber_table['RA'], matched_fiber_table['ra'])

In [ ]:
plt.scatter( matched_fiber_table['DEC'], matched_fiber_table['dec'])